# Data loading 
Next cells are used to load the data we need.

In [1]:
import os

os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1

import pandas as pd
import numpy as np 
import scipy.sparse as sps
import sys
import matplotlib.pyplot as plt
sys.path.append('../../')


In [2]:
#data which contains users (row), items(col) and implicit interaction (data)
dataset = pd.read_csv('../data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


# Data pre-processing
Pre-processing of data to check for missing users, items ecc...

In [3]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

(7947, 25975)

In [4]:
ICM_df = pd.read_csv('../data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [5]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [6]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [7]:
from MatrixFactorization.IALSRecommender_implicit import IALSRecommender_implicit

#Trial 22 finished with value: 0.07502862236249128 and parameters: {'n_factors': 867, 'regularization': 0.7451746230098858, 'iterations': 74, 'weight': 0.5512468009549221}. Best is trial 22 with value: 0.07502862236249128.[0m
#printing self map vector:  [0.07496821 0.07736574 0.07275191]
#printing MAP:  0.07502862236249128

#current best parameters. Tune again near these values
#printing MAP:  0.07395163955000743
#{'n_factors': 767, 'regularization': 0.6762816208423306, 'iterations': 69, 'weight': 0.5686851029710857}

#{'n_factors': 779, 'regularization': 0.41829264231590485, 'iterations': 72, 'weight': 0.533979793083854}

URM_train_ICM_all_IALS = sps.vstack([URM_all*(1-0.5293971458364172), ICM_all.T*0.5293971458364172])
recommender_IALS = IALSRecommender_implicit(URM_train_ICM_all_IALS)
recommender_IALS.fit(n_factors = 768, regularization = 0.4489004525533907, iterations=76)

Recommender_Base_Class: URM Detected 2 (0.01 %) cold users.


In [8]:
from GraphBased.RP3betaRecommender import RP3betaRecommender
#already 3 crossed validated these values
URM_train_ICM_all_RP3 = sps.vstack([URM_all, ICM_all.T])
recommender_rp3Beta = RP3betaRecommender(URM_train_ICM_all_RP3)
recommender_rp3Beta.fit(topK=207, alpha = 0.36128728066686633, beta=0.26052832791376884, implicit = False) 
#recommender_rp3Beta.URM_train = URM_train

RP3betaRecommender: URM Detected 2 (0.01 %) cold users.


In [9]:
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

URM_train_ICM_all_SLIM = sps.vstack([URM_all*(1-0.421), ICM_all.T*0.421])
recommender_SLIM = SLIMElasticNetRecommender(URM_train_ICM_all_SLIM)
recommender_SLIM.fit(topK=996, alpha = 0.000182, l1_ratio = 4.459e-06)
recommender_SLIM.W_sparse.data = np.power(recommender_SLIM.W_sparse.data, 0.8546)

SLIMElasticNetRecommender: URM Detected 2 (0.01 %) cold users.


C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0013661469565704465, tolerance: 0.0008219736628234386
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0009991888655349612, tolerance: 0.0005707712844014168
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0002764826640486717, tolerance: 0.00011493572674226016
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\en

SLIMElasticNetRecommender: Processed 4275 ( 16.46% ) in 5.00 min. Items per second: 14.24


C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0014586295001208782, tolerance: 0.0006375000230036676
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.014916838146746159, tolerance: 0.003089803270995617
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.01915721967816353, tolerance: 0.0028581060469150543
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my

SLIMElasticNetRecommender: Processed 8511 ( 32.77% ) in 10.00 min. Items per second: 14.18


...

SLIMElasticNetRecommender: Processed 21132 ( 81.36% ) in 25.00 min. Items per second: 14.09


C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00013123977987561375, tolerance: 0.00010085513349622488
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00036053601070307195, tolerance: 0.00017302924243267626
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.04919232055544853, tolerance: 0.0038821203634142876
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\e

SLIMElasticNetRecommender: Processed 25229 ( 97.13% ) in 30.00 min. Items per second: 14.01


C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.676663466962054e-05, tolerance: 3.487366120680235e-05
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.11030993610620499, tolerance: 0.008016743697226048
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.05598107725381851, tolerance: 0.0018809903413057327
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_

SLIMElasticNetRecommender: Processed 25975 ( 100.00% ) in 30.91 min. Items per second: 14.00


In [10]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython

URM_train_ICM_all_SLIM = sps.vstack([URM_all*(1- 0.6229636472829045), ICM_all.T* 0.6229636472829045])
recommender_SLIM_BPR = SLIM_BPR_Cython(URM_train_ICM_all_SLIM, recompile_cython=False, verbose=False)
recommender_SLIM_BPR.fit(epochs=87, sgd_mode= "adam", learning_rate= 6.12252603066224e-06, topK = 805, batch_size = 35, lambda_i=3.0399585540912118e-05, lambda_j=0.00012176512055401498 )

Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 87. Elapsed time 0.11 sec
SLIM_BPR_Recommender: Epoch 2 of 87. Elapsed time 0.20 sec
SLIM_BPR_Recommender: Epoch 3 of 87. Elapsed time 0.28 sec
SLIM_BPR_Recommender: Epoch 4 of 87. Elapsed time 0.37 sec
SLIM_BPR_Recommender: Epoch 5 of 87. Elapsed time 0.45 sec
SLIM_BPR_Recommender: Epoch 6 of 87. Elapsed time 0.54 sec
SLIM_BPR_Recommender: Epoch 7 of 87. Elapsed time 0.62 sec
SLIM_BPR_Recommender: Epoch 8 of 87. Elapsed time 0.71 sec
SLIM_BPR_Recommender: Epoch 9 of 87. Elapsed time 0.80 sec
SLIM_BPR_Recommender: Epoch 10 of 87. Elapsed time 0.89 sec
SLIM_BPR_Recommender: Epoch 11 of 87. Elapsed time 0.98 sec
SLIM_BPR_Recommender: Epoch 12 of 87. Elapsed time 1.07 sec
SLIM_BPR_Recommender: Epoch 13 of 87. Elapsed time 1.16 sec
SLIM_BPR_Recommender: Epoch 14 of 87. Elapsed time 1.26 sec
SLIM_BPR_Recommender: Epoch 15 of 87. Elapsed time 1.35 sec
SLIM_BPR_Recommender: Epoch 16 of 87. Elapsed time 1.43

In [11]:
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
URM_train_ICM_all = sps.vstack([URM_all, ICM_all.T])
recommender_itemKNN = ItemKNNCFRecommender(URM_train_ICM_all)
recommender_itemKNN.fit(shrink=600, topK=248, feature_weighting = "TF-IDF", similarity='cosine')


ItemKNNCFRecommender: URM Detected 2 (0.01 %) cold users.
Similarity column 25975 ( 100 % ), 3921.14 column/sec, elapsed time 0.11 min


In [12]:
#group 0 recommender 
#{'alpha': 0.4529884904780542, 'beta': 0.8311638292609006, 'gamma': 0.12038869638040192, 'phi': 0.8557562262354292, 'omega': 0.13896818557599339}

from KNN.ItemKNNScoresHybridNRecommender import ItemKNNScoresHybridNRecommender
URM_train_ICM_all = sps.vstack([URM_all, ICM_all.T])
hybrid = ItemKNNScoresHybridNRecommender(URM_train_ICM_all, [recommender_IALS,recommender_rp3Beta,recommender_SLIM,recommender_SLIM_BPR,recommender_itemKNN], 5)
hybrid.fit([0.4529884904780542,   0.8311638292609006, 0.12038869638040192, 0.8557562262354292,0.13896818557599339])
#evaluator_validation.evaluateRecommender(hybrid)

ItemKNNScoresHybridNRecommender: URM Detected 2 (0.01 %) cold users.
number of recommenders: 5


In [13]:
test_users = pd.read_csv('../data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
7939,7942
7940,7943
7941,7944
7942,7945


In [14]:
user_id = test_users['user_id']
recommendations = hybrid.recommend(user_id,cutoff = 10)

In [15]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])
    
print(len(recommendations))

test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])

test_users
test_users.to_csv('submission.csv', index=False)


7944
